In [1]:
from random import randint, uniform, choice
from util import generate_problem, verify_witness

INF = 1e9

## Firstly we need a solver for the STP problem.

note here we are given a distance graph
(the generation is done in the genetic part, so we can reuse memory and save time when an interval selection is changed.)

In [2]:
def discrete_graph(N):
    mat = [[INF for _ in range(N)] for _ in range(N)]
    for i in range(N):
        mat[i][i] = 0
    return mat

def generate_d_graph(graph):
    N = len(graph)
    E = discrete_graph(N)
    
    for i in range(N):
        E[i][i] = 0 # redundant

    for i in range(N):
        for j in range(N):
            E[i][j] = graph[i][j]

    for k in range(N):
        for i in range(N):
            for j in range(N):
                E[i][j] = min(E[i][j], E[i][k] + E[k][j])

    return E

def get_min_sol(d_graph):
    return [-d_graph[i][0] for i in range(len(d_graph))]

def consistent(d_graph):
    for i in range(len(d_graph)):
        if d_graph[i][i] < 0: return False
    return True

def solve_stp(graph):
    ''' graph is the constraint adjacency matrix '''
    
    d_graph = generate_d_graph(graph)
    
    return consistent(d_graph), get_min_sol(d_graph)

## The genetic part:

In [122]:
def update_graph(gene, T):
    # set up graph using index 0 everywhere
    # so later we can change interval in O(1) time
    # (saves memory and time, and this same approach is especially beneficial in meta_walk)
    
    interval_selection = gene[0]
    graph = gene[1]
    gene[2] = None # unset
    
    for i, constr in enumerate(T):
        interval_index = interval_selection[i]
        
        i, j = constr['i'], constr['j']
        interval = constr['intervals'][interval_index]
        
        graph[i][j] = interval[1]
        graph[j][i] = -interval[0]

def update_graph_at_constraint(gene, constraint_index):
    ''' update_graph but updates only one constraint '''
    
    interval_selection = gene[0]
    graph = gene[1]
    gene[2] = None # unset
    
    interval_index = interval_selection[constraint_index]
    constr = T[constraint_index]
    
    i, j = constr['i'], constr['j']
    interval = constr['intervals'][interval_index]

    graph[i][j] = interval[1]
    graph[j][i] = -interval[0]

def random_gene(T):
    num_variables = max([max(t['i'], t['j']) for t in T])
    interval_selection = [randint(0, len(constr['intervals'])-1) for constr in T]
    gene = [interval_selection, discrete_graph(num_variables+1), None]
    update_graph(gene, T)
    
    return gene
    
def walk_gene(gene, T):
    # ignore all that have no choice
    # create list of plausible indices and filter it.
    candidate_indices = [j for j in range(len(T))]
    candidate_indices = list(filter(
        lambda x: len(T[x]['intervals']) > 1,
        candidate_indices
    ))
    
    if candidate_indices:
        idx = choice(candidate_indices)
        gene[0][idx] = randint(0, len(T[idx]['intervals'])-1)
        update_graph_at_constraint(gene, idx)
    
def evaluate(genes, T):
    for g in genes:
        d_graph = generate_d_graph(g[1]) # costly op.
        g[2] = verify_witness(get_min_sol(d_graph), T)
    return genes

def fitness(gene, T):
    # make sure that gene[2] is set to failed constraints
    # otherwise this will not work ideally (or fail if it's None.)
    return -len(gene[2])

def select(genes, retainment_ratio, T):
    genes.sort(key=lambda g: -fitness(g, T))
    return genes[: int(len(genes)*retainment_ratio+1) ]



def crossover(genes, gene_pool_size, T):
    num_variables = max([max(t['i'], t['j']) for t in T])
    while len(genes) < gene_pool_size:
        i = randint(0, len(genes)-1)
        j = randint(0, len(genes)-1)
        cross_index = randint(1, len(genes[0])-1)
        
        g = [
            genes[i][0][:cross_index] + genes[j][0][cross_index:],
            discrete_graph(num_variables+1),
            None
        ]
        
        genes.append(g)
        
    return genes

def mutate(T, genes, mutation_chance):
    for g in genes:
        if uniform(0, 1) < mutation_chance:
            walk_gene(g, T)
            
    return genes

def meta_genetic(T,
            gene_pool_size,
            retainment_ratio,
            mutation_chance,
            max_iterations):
    
    genes = [random_gene(T) for i in range(gene_pool_size)]
    
    for it in range(max_iterations):
        genes = evaluate(genes, T)
        genes = select(genes, retainment_ratio, T)
        genes = crossover(genes, gene_pool_size, T)
        genes = mutate(T, genes, mutation_chance)
    
    genes = evaluate(genes, T)
    best_gene = select(genes, 1, T)[0] # to sort genes such that first is best
    best_gene_failed = best_gene[2]
    print('best gene:', best_gene[0])
    print('constraints failed:', len(best_gene_failed), 'out of:', len(T)) 
    print('failed constraints:', best_gene_failed)

In [153]:
#T = generate_problem(variables=15)
meta_genetic(
    T, 100, 0.1, 0.1, 10
)

best gene: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 0, 0, 2, 0, 1, 1, 2, 0, 2, 2, 1, 0, 1, 2, 0, 0, 3, 0, 1, 0, 0, 2, 1]
constraints failed: 13 out of: 41
failed constraints: [{'i': 0, 'j': 12, 'intervals': [(-100, 100)]}, {'i': 0, 'j': 13, 'intervals': [(-100, 100)]}, {'i': 0, 'j': 14, 'intervals': [(-100, 100)]}, {'i': 1, 'j': 13, 'intervals': [(-85, 93)]}, {'i': 1, 'j': 14, 'intervals': [(-86, -59), (-53, -39), (-39, 4), (21, 80)]}, {'i': 5, 'j': 11, 'intervals': [(-87, -64), (-48, -40), (-10, 56), (70, 85)]}, {'i': 6, 'j': 11, 'intervals': [(-92, -51), (-30, -24), (2, 94)]}, {'i': 8, 'j': 13, 'intervals': [(-30, 19), (20, 30), (41, 79), (88, 96)]}, {'i': 8, 'j': 14, 'intervals': [(-92, -84)]}, {'i': 10, 'j': 12, 'intervals': [(-12, 42), (43, 51), (64, 76), (79, 88)]}, {'i': 10, 'j': 14, 'intervals': [(5, 63)]}, {'i': 11, 'j': 12, 'intervals': [(-79, -28), (-14, 3), (32, 42), (58, 84)]}, {'i': 12, 'j': 14, 'intervals': [(-98, -78), (-74, -49), (2, 8), (10, 19), (26, 10

In [8]:
T = generate_problem()
gene = random_gene(T)
gene

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 1, 2, 1],
 [[100, 100, 100, 100, 100, 100, 100, 100, 100, 100],
  [100,
   0,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   -13,
   1000000000.0,
   1000000000.0,
   24,
   1000000000.0],
  [100,
   1000000000.0,
   0,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   55,
   1000000000.0,
   1000000000.0,
   -24],
  [100,
   1000000000.0,
   1000000000.0,
   0,
   1000000000.0,
   76,
   1000000000.0,
   3,
   1000000000.0,
   1000000000.0],
  [100,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   0,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   1000000000.0],
  [100,
   48,
   1000000000.0,
   -29,
   1000000000.0,
   0,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   1000000000.0],
  [100,
   1000000000.0,
   -51,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   0,
   1000000000.0,
   1000000000.0,
   1000000000.0],
  [100,
   1000000000.0,
   1000000000.0,
   6,
   1000000000.0,
   10000000

In [9]:
walk_gene(gene, T)

In [10]:
gene

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 1, 2, 1],
 [[100, 100, 100, 100, 100, 100, 100, 100, 100, 100],
  [100,
   0,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   -13,
   1000000000.0,
   1000000000.0,
   -76,
   1000000000.0],
  [100,
   1000000000.0,
   0,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   55,
   1000000000.0,
   1000000000.0,
   -24],
  [100,
   1000000000.0,
   1000000000.0,
   0,
   1000000000.0,
   76,
   1000000000.0,
   3,
   1000000000.0,
   1000000000.0],
  [100,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   0,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   1000000000.0],
  [100,
   48,
   1000000000.0,
   -29,
   1000000000.0,
   0,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   1000000000.0],
  [100,
   1000000000.0,
   -51,
   1000000000.0,
   1000000000.0,
   1000000000.0,
   0,
   1000000000.0,
   1000000000.0,
   1000000000.0],
  [100,
   1000000000.0,
   1000000000.0,
   6,
   1000000000.0,
   1000000